In [1]:
!pip install tabpfn --no-index --find-links=file:///kaggle/input/pip-packages-icr/pip-packages
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

Looking in links: file:///kaggle/input/pip-packages-icr/pip-packages
Processing /kaggle/input/pip-packages-icr/pip-packages/tabpfn-0.1.9-py3-none-any.whl


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,normalize
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import xgboost
import inspect
from collections import defaultdict
from tabpfn import TabPFNClassifier
from catboost import CatBoostClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from sklearn.utils import class_weight
import optuna
from functools import partial
import logging
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import time
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV

from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score, log_loss
from sklearn.svm import SVC

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from numpy.random import seed
seed(42) 
import random as pyrandom
pyrandom.seed(42)

In [4]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
sample = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')

In [5]:
def get_traindf():
    train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv') 
    greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
    
    rows=[102,194,203,292,356,360,372,457,458,586] 
    train['Correction'] = 0
    train.loc[rows,'Correction']=1 

    first_category = train.EJ.unique()[0]
    train.EJ = train.EJ.eq(first_category).astype('int')
    test.EJ = test.EJ.eq(first_category).astype('int')

    from datetime import datetime
    times = greeks.Epsilon.copy()
    times[greeks.Epsilon != 'Unknown'] = greeks.Epsilon[greeks.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
    times[greeks.Epsilon == 'Unknown'] = np.nan

    train_pred_and_time = pd.concat((train, times), axis=1)
    return train_pred_and_time

In [6]:
def get_traindf2():
    train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv') 
    greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
    
    rows=[102,194,203,292,356,360,372,457,458,586] 
    train['Correction'] = 0
    train.loc[rows,'Correction']=1 

#     first_category = train.EJ.unique()[0]
#     train.EJ = train.EJ.eq(first_category).astype('int')
#     test.EJ = test.EJ.eq(first_category).astype('int')

    from datetime import datetime
    times = greeks.Epsilon.copy()
    times[greeks.Epsilon != 'Unknown'] = greeks.Epsilon[greeks.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
    times[greeks.Epsilon == 'Unknown'] = np.nan

    train_pred_and_time = pd.concat((train, times), axis=1)
    return train_pred_and_time

In [7]:
# def get_preprocessed(train,test):
# #     train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv') 
# #     test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
#     predictor_columns = [n for n in train.columns if n != 'Class' and n != 'Id']
    
#     greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')

#     first_category = train.EJ.unique()[0]
#     train.EJ = train.EJ.eq(first_category).astype('int')
#     test.EJ = test.EJ.eq(first_category).astype('int')

#     from datetime import datetime
#     times = greeks.Epsilon.copy()
#     times[greeks.Epsilon != 'Unknown'] = greeks.Epsilon[greeks.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
#     times[greeks.Epsilon == 'Unknown'] = np.nan

#     train_pred_and_time = pd.concat((train, times), axis=1)    
    
#     test_predictors = test[predictor_columns]
#     first_category = test_predictors.EJ.unique()[0]
#     test_predictors.EJ = test_predictors.EJ.eq(first_category).astype('int')
#     test_predictors['Epsilon'] = train_pred_and_time.Epsilon.max() + 1
    
# #     test_pred_and_time = np.concatenate((test_predictors, np.zeros((len(test_predictors), 1)) + train_pred_and_time.Epsilon.max() + 1), axis=1)
# #     train_pred_and_time.drop(['Class','Id'],inplace=True, axis=1)
#     train_pred_and_time = feature(train_pred_and_time)
#     test_predictors = feature(test_predictors)
#     df = pd.concat([train_pred_and_time,test_predictors])
#     bq_median = df['BQ'].median()
#     el_median = df['EL'].median() 
    
#     train_pred_and_time['BQ']=train_pred_and_time['BQ'].fillna(bq_median)
#     train_pred_and_time['EL']=train_pred_and_time['EL'].fillna(el_median)
    
#     test_predictors['BQ']=test_predictors['BQ'].fillna(bq_median)
#     test_predictors['EL']=test_predictors['EL'].fillna(el_median)
    
   
    
# #     imputer_knn = KNNImputer(n_neighbors=3) #, weights="distance")
# #     sc=StandardScaler()
    
# #     imputer_knn.fit(df)
# #     sc.fit(df)
    
#     return train_pred_and_time,test_predictors

In [8]:
# test2 = train.sample(frac=0.3,random_state=42)
# test = test2.copy(deep=True)
# # test.drop(['Id','Class'],inplace=True, axis=1)
# y_true = test2['Class']
# test2.drop(['Class'],inplace=True, axis=1)

In [9]:
train.columns[train.isnull().any()]

Index(['BQ', 'CB', 'CC', 'DU', 'EL', 'FC', 'FL', 'FS', 'GL'], dtype='object')

In [10]:
# train['dupind'aggregate] = [1 if train.loc[:,['AR', 'AY', 'BZ', 'DF', 'DV']].duplicated() else 0]

# train['dupind'] = train.loc[:, ['AR', 'AY', 'BZ', 'DF', 'DV']].duplicated().astype(int)
# test['dupind']= test.loc[:, ['AR', 'AY', 'BZ', 'DF', 'DV']].duplicated().astype(int)


In [11]:
# bq_median = train['BQ'].median()
# el_median = train['EL'].median()
# train['BQ']=train['BQ'].fillna(bq_median)
# train['EL']=train['EL'].fillna(el_median)

In [12]:
first_category = train.EJ.unique()[0]
train.EJ = train.EJ.eq(first_category).astype('int')
test.EJ = test.EJ.eq(first_category).astype('int')

In [13]:
def feature(data):
    X=data.copy(deep=True)
    X['out_GL'] = 0
    X.loc[X['GL']<1,'out_GL'] = X.loc[X['GL']<1,'GL'].map(lambda x : x-X.loc[X['GL']<1,'GL'].mean())
    X.loc[X['GL']>1.5,'out_GL'] = X.loc[X['GL']>1.5,'GL'].map(lambda x : x-X.loc[X['GL']>1.5,'GL'].mean())
    X.out_GL = X.out_GL.astype('float')
    X['DA*CS'] = np.log(X.DA*2 / X.CS**0.5)
#     X['dupind1'] = X.loc[:, ['AR', 'AY', 'BZ', 'DF', 'DV']].duplicated().astype(int)
    X['dupind1'] = X.loc[:, ['AR', 'AY', 'BZ', 'DF', 'DV']].duplicated(keep=False).astype(int)

    X['dupind2'] = 1-X['dupind1']
    return X

In [14]:
# def feature(data):
#     X=data.copy(deep=True)
#     X['out_GL'] = 0
#     X.loc[X['GL']<1,'out_GL'] = X.loc[X['GL']<1,'GL'].map(lambda x : x-X.loc[X['GL']<1,'GL'].mean())
#     X.loc[X['GL']>1.5,'out_GL'] = X.loc[X['GL']>1.5,'GL'].map(lambda x : x-X.loc[X['GL']>1.5,'GL'].mean())
#     X.out_GL = X.out_GL.astype('float')
#     X['DA*CS'] = np.log(X.DA*2 / X.CS**0.5)
#     return X

In [15]:
# def feature1(data):
#     X=data.copy(deep=True)
#     X['out_GL'] = 0
#     X.loc[X['GL']<1,'out_GL'] = X.loc[X['GL']<1,'GL'].map(lambda x : x-X.loc[X['GL']<1,'GL'].mean())
#     X.loc[X['GL']>1.5,'out_GL'] = X.loc[X['GL']>1.5,'GL'].map(lambda x : x-X.loc[X['GL']>1.5,'GL'].mean())
#     X.out_GL = X.out_GL.astype('float')
# #     X['DA*CS'] = np.log(X.DA*2 / X.CS**0.5)
#     return X

In [16]:
from sklearn.model_selection import KFold as KF, GridSearchCV
 
cv_inner = KF(n_splits = 5, shuffle=True, random_state=42)

In [17]:
def balanced_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # calculate the weights for each class to balance classes
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    # calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # calculate the summed log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    # calculate the weighted summed logarithmic loss
    # (factgor of 2 included to give same result as LL with balanced input)
    balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    # return the average log loss
    return balanced_log_loss/(N_0+N_1)

In [18]:
class Ensemble():
    def __init__(self):
        self.imputer = SimpleImputer(missing_values=np.nan, strategy='median')

        self.classifiers =[xgboost.XGBClassifier(n_estimators=100,max_depth=3,learning_rate=0.2,subsample=0.9,colsample_bytree=0.85),
                           xgboost.XGBClassifier(),
                           TabPFNClassifier(N_ensemble_configurations=24),
                          TabPFNClassifier(N_ensemble_configurations=64)]
    
    def fit(self,X,y):
        y = y.values
        unique_classes, y = np.unique(y, return_inverse=True)
        self.classes_ = unique_classes
        first_category = X.EJ.unique()[0]
        X.EJ = X.EJ.eq(first_category).astype('int')
        X = self.imputer.fit_transform(X)
#         X = normalize(X,axis=0)
        for classifier in self.classifiers:
            if classifier==self.classifiers[2] or classifier==self.classifiers[3]:
                classifier.fit(X,y,overwrite_warning =True)
            else :
                classifier.fit(X, y)
     
    def predict_proba(self, x):
        x = self.imputer.transform(x)
#         x = normalize(x,axis=0)
        probabilities = np.stack([classifier.predict_proba(x) for classifier in self.classifiers])
        averaged_probabilities = np.mean(probabilities, axis=0)
        class_0_est_instances = averaged_probabilities[:, 0].sum()
        others_est_instances = averaged_probabilities[:, 1:].sum()
        # Weighted probabilities based on class imbalance
        new_probabilities = averaged_probabilities * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(averaged_probabilities.shape[1])]])
        return new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1) 

In [19]:
def ScoreMetric1(y_true, y_pred): 
    
    ytrue = [0 if val == 0 else 1 for val in y_true]
    
     
    probabilities = np.concatenate((y_pred[:,:1], np.sum(y_pred[:,1:], 1, keepdims=True)), axis=1)
    p0 = probabilities[:,:1] 
     
    nc = np.bincount(ytrue);
    return log_loss(ytrue, 1-p0, sample_weight = 1 / nc[ytrue], eps=1e-15) 

In [20]:
# ytst = greeks.Alpha.copy(deep=True)

# le = LabelEncoder()
# y_meta1 = le.fit_transform(ytst)
# print(le.classes_)      

# xz = train_pred_and_time.copy(deep=True)
# xz.drop(['Id','Class'],inplace=True, axis=1)

# impute = SimpleImputer(missing_values=np.nan, strategy='median')
# xz1 = impute.fit_transform(xz)  


# sc=StandardScaler()
# xz2=sc.fit_transform(xz1)

# param_grid = {                      
                                   
#             'learning_rate': [0.1,0.2],
# #             'scale_pos_weight': [4.71],
    
#             'lambda': [0.01,0.001],
#             'alpha': [0.0001,0.00001],
    
# #             'lambda': [0.01,0.1,0.001],
# #             'alpha': [0.5,0.03,0.00001],
#             'subsample': [0.8],
#             'colsample_bytree': [0.7,0.9],
#             'max_depth': [10,20,100],
#             'min_child_weight': [5,10],
# #             'class_weight':['balanced']
# #             'eta': 2.09477807126539E-06,
# #             'gamma': 0.000847289463422307,
# #             'grow_policy': 'depthwise',
# #             'n_jobs': -1,
# #             'objective': 'binary:logistic',
# #             'eval_metric': 'logloss',
# #             'verbosity': 0,
# #             'random_state': 42) 
#              }

 
# grid_search = GridSearchCV(xgb, param_grid, cv=5, scoring=make_scorer(ScoreMetric1, needs_proba=True,greater_is_better=False))


# grid_search.fit(xz2, y_meta1)

# # Print the best hyperparameters found
# print("Best Hyperparameters: ", grid_search.best_params_)

# # Evaluate the best model on your data
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(xz2)

# # Print classification report or other evaluation metrics
# print("Classification Report:")
# print(classification_report(y_meta1, y_pred))

# #Best Hyperparameters:  {'alpha': 0.0001, 'colsample_bytree': 0.7, 'lambda': 0.01, 
# # 'learning_rate': 0.2, 'max_depth': 20, 'min_child_weight': 10, 'subsample': 0.8}

# # {'alpha': 0.0001, 'class_weight': 'balanced', 'colsample_bytree': 0.7, 'lambda': 0.01, 
# #  'learning_rate': 0.2, 'max_depth': 20, 'min_child_weight': 10, 'subsample': 0.8}
    

In [21]:
# param_grid = {
#                'colsample_bytree'  : [0.4,0.7], #[0.7,0.9]
#                'learning_rate'     : [0.2,0.1], #[0.10,0.2]
#                'max_depth'         : [5,10],
#                'min_child_samples' : [5,10],                
#                'num_leaves'        : [40,60], #[40,100]
#                'reg_alpha'         : [0.001,0.005,0.1,0.2],
#                'reg_lambda'        : [0.1,0.65,1],
#                'subsample'         : [0.65,0.8] #[0.65, 0.8]
    
#                                   }

 
# grid_search = GridSearchCV(LGBMClassifier(), param_grid, cv=5, scoring=make_scorer(ScoreMetric1, needs_proba=True,greater_is_better=False))

# # ros = RandomOverSampler(random_state=42)
# # x_train1,y_train1 = ros.fit_resample(xz2, y_meta1) 
# # grid_search.fit(x_train1, y_train1)

# grid_search.fit(xz2, y_meta1)

# # Print the best hyperparameters found
# print("Best Hyperparameters: ", grid_search.best_params_)

# # Evaluate the best model on your data
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(xz2)

# # Print classification report or other evaluation metrics
# print("Classification Report:")
# print(classification_report(y_meta1, y_pred))

# # {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_samples': 10,
# # 'num_leaves': 40, 'reg_alpha': 0.001, 'reg_lambda': 0.65, 'subsample': 0.8}

# # {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_samples': 10, 'num_leaves': 40, 'reg_alpha': 0.001, 'reg_lambda': 0.65, 'subsample': 0.8}
    
# #{'colsample_bytree': 0.4, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_samples': 5, 
# # 'num_leaves': 40, 'reg_alpha': 0.005, 'reg_lambda': 0.65, 'subsample': 0.65}


# #  {'colsample_bytree': 0.4, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_samples': 10, 'num_leaves': 40, 'reg_alpha': 0.2, 'reg_lambda': 0.65, 'subsample': 0.65}

In [22]:
# param_grid = {
#                C : [0.1,10,20],
#     penalty :[ 'l1','l2'] , 
#     multi_class: ['ovr'],
#      solver :['liblinear','lbfgs'],
#     max_iter : [300,2000,5000]
    
#                                   }

 
# grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring=make_scorer(ScoreMetric1, needs_proba=True,greater_is_better=False))

# # ros = RandomOverSampler(random_state=42)
# # x_train1,y_train1 = ros.fit_resample(xz2, y_meta1) 
# # grid_search.fit(x_train1, y_train1)

# grid_search.fit(xz2, y_meta1)

# # Print the best hyperparameters found
# print("Best Hyperparameters: ", grid_search.best_params_)

# # Evaluate the best model on your data
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(xz2)

# # Print classification report or other evaluation metrics
# print("Classification Report:")
# print(classification_report(y_meta1, y_pred))

# # {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_samples': 10,
# # 'num_leaves': 40, 'reg_alpha': 0.001, 'reg_lambda': 0.65, 'subsample': 0.8}

# # {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_samples': 10, 'num_leaves': 40, 'reg_alpha': 0.001, 'reg_lambda': 0.65, 'subsample': 0.8}
    
# #{'colsample_bytree': 0.4, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_samples': 5, 
# # 'num_leaves': 40, 'reg_alpha': 0.005, 'reg_lambda': 0.65, 'subsample': 0.65}

In [23]:
# ytst = greeks.Alpha.copy(deep=True)

# le = LabelEncoder()
# y_meta1 = le.fit_transform(ytst)
# print(le.classes_)      

# xz = train_pred_and_time.copy(deep=True)
# xz.drop(['Id','Class'],inplace=True, axis=1)

# impute = SimpleImputer(missing_values=np.nan, strategy='median')
# xz1 = impute.fit_transform(xz)  


# sc=StandardScaler()
# xz2=sc.fit_transform(xz1)

# param_grid = {'C' : [0.1,0.5],
#     'penalty' :[ 'l1','l2'] , 
#     'multi_class': ['ovr'],
#      'solver' :['liblinear','lbfgs'],
#     'class_weight': ['balanced',None],
#     'max_iter' : [300,2000,5000]
    
#                                   }

 
# grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring=make_scorer(ScoreMetric1, needs_proba=True,greater_is_better=False))


# grid_search.fit(xz2, y_meta1)

# # Print the best hyperparameters found
# print("Best Hyperparameters: ", grid_search.best_params_)

# # Evaluate the best model on your data
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(xz2)

# # Print classification report or other evaluation metrics
# print("Classification Report:")
# print(classification_report(y_meta1, y_pred))
    

# # {'C': 0.5, 'class_weight': 'balanced', 'max_iter': 300, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'liblinear'}

In [24]:
# #  cat_params = {
# #         'learning_rate': 0.005, 
# #         'iterations': num_boost_round, 
# #         'depth': 4, # 
# #         'colsample_bylevel': 0.50,
# #         'subsample': 0.80,
# #         'l2_leaf_reg': 3, # 3, 30
# #         'random_seed': seed,
# #         'auto_class_weights': 'Balanced'
# #         # 'task_type':'GPU', 
# #     }
    
    
# ytst = greeks.Alpha.copy(deep=True)

# le = LabelEncoder()
# y_meta1 = le.fit_transform(ytst)
# print(le.classes_)      

# xz = train_pred_and_time.copy(deep=True)
# xz = feature(xz)
# xz.drop(['Id','Class'],inplace=True, axis=1)

# impute = SimpleImputer(missing_values=np.nan, strategy='median')
# xz1 = impute.fit_transform(xz)  

# sc=StandardScaler()
# xz2=sc.fit_transform(xz1)

# param_grid = cat_params = {
#         'learning_rate': [0.01,0.05] ,         
#         'depth': [4, 7],
#         'colsample_bylevel': [0.50],
# #         'subsample': [0.80],
#         'l2_leaf_reg': [3],# 3, 30
#     'verbose' :[False],
#         'auto_class_weights': ['Balanced']
#     }

# grid_search = GridSearchCV( CatBoostClassifier(), param_grid, cv=5, scoring=make_scorer(ScoreMetric1, needs_proba=True,greater_is_better=False))

# grid_search.fit(xz2, y_meta1)

# # Print the best hyperparameters found
# print("Best Hyperparameters: ", grid_search.best_params_)

# # Evaluate the best model on your data
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(xz2)

# # Print classification report or other evaluation metrics
# print("Classification Report:")
# print(classification_report(y_meta1, y_pred))
    


# # Best Hyperparameters:  {'C': 0.01, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1}

# # Best Hyperparameters:  {'C': 0.01, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1}
    
    
# # Best Hyperparameters:  {'auto_class_weights': 'Balanced', 'colsample_bylevel': 0.5, 'depth': 4, 'l2_leaf_reg': 3, 'learning_rate': 0.01, 'verbose': False}
# # Classification Report:
# #               precision    recall  f1-score   support

# #            0       1.00      0.94      0.97       509
# #            1       0.71      1.00      0.83        61
# #            2       0.95      1.00      0.97        18
# #            3       0.94      1.00      0.97        29

# #     accuracy                           0.95       617
# #    macro avg       0.90      0.99      0.94       617
# # weighted avg       0.97      0.95      0.96       617

# # Best Hyperparameters:  {'auto_class_weights': 'Balanced', 'colsample_bylevel': 0.5, 'depth': 4, 'l2_leaf_reg': 3, 'learning_rate': 0.05, 'verbose': False}
# # Classification Report:
# #               precision    recall  f1-score   support

# #            0       1.00      1.00      1.00       509
# #            1       1.00      1.00      1.00        61
# #            2       1.00      1.00      1.00        18
# #            3       1.00      1.00      1.00        29

# #     accuracy                           1.00       617
# #    macro avg       1.00      1.00      1.00       617
# # weighted avg       1.00      1.00      1.00       617


In [25]:


# ytst = greeks.Alpha.copy(deep=True)

# le = LabelEncoder()
# y_meta1 = le.fit_transform(ytst)
# print(le.classes_)      

# xz = train_pred_and_time.copy(deep=True)
# xz.drop(['Id','Class'],inplace=True, axis=1)

# impute = SimpleImputer(missing_values=np.nan, strategy='median')
# xz1 = impute.fit_transform(xz)  


# sc=StandardScaler()
# xz2=sc.fit_transform(xz1)

# param_grid = {'C' : [10,15,20],
#     'gamma' :[ 'auto','scale'],
#      'kernel': ['rbf','linear','poly','sigmoid'], #‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’]
#      'decision_function_shape':['ovo'],
#     'class_weight': ['balanced'],
#     'max_iter' : [-1]
    
#                                   }

 
# grid_search = GridSearchCV( svc, param_grid, cv=5, scoring=make_scorer(ScoreMetric1, needs_proba=True,greater_is_better=False))


# grid_search.fit(xz2, y_meta1)

# # Print the best hyperparameters found
# print("Best Hyperparameters: ", grid_search.best_params_)

# # Evaluate the best model on your data
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(xz2)

# # Print classification report or other evaluation metrics
# print("Classification Report:")
# print(classification_report(y_meta1, y_pred))
    


# # Best Hyperparameters:  {'C': 0.01, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1}

# # Best Hyperparameters:  {'C': 0.01, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1}

In [26]:
# xgb1 = xgboost.XGBClassifier(alpha=0.0005, base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=0.4, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, reg_lambda=0.01, learning_rate=0.4,
#               max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=None, max_leaves=None,
#               min_child_weight=1,   monotone_constraints=None,
#               n_estimators=100, n_jobs=-1, num_parallel_tree=None) #,tree_method = 'gpu_hist' )  

xgb = xgboost.XGBClassifier(**{'alpha': 0.0001, 'colsample_bytree': 0.7, 'lambda': 0.01, 
                               'learning_rate': 0.2, 'max_depth': 20, 
                               'min_child_weight': 10, 'subsample': 0.8})

# xgbparams = {'alpha': 0.0001, 'colsample_bytree': 0.7, 'lambda': 0.01, 
#                                'learning_rate': 0.2, 'max_depth': 20, 
#                                'min_child_weight': 10, 'subsample': 0.8}
         

# cb1 = CatBoostClassifier(verbose=False,auto_class_weights='Balanced')

# cb2 = CatBoostClassifier(learning_rate=0.005, 
#     verbose= False,
     
#     depth =4, # 
#     colsample_bylevel= 0.50,
# #     subsample= 0.80,
#     l2_leaf_reg= 3, # 3, 30
     
#     auto_class_weights='Balanced')

# cb = CatBoostClassifier(**cat_params)  

# cat_params = {
#     'learning_rate': 0.005, 
#     'verbose' : False,
     
#     'depth': 4, # 
#     'colsample_bylevel': 0.50,
#     'subsample': 0.80,
#     'l2_leaf_reg': 3, # 3, 30
#     'random_seed': seed,
#     'auto_class_weights': 'Balanced'
#     # 'task_type':'GPU', 
# }

cb=CatBoostClassifier(**{'auto_class_weights': 'Balanced', 'colsample_bylevel': 0.5, 'depth': 4, 
                         'l2_leaf_reg': 3, 'learning_rate': 0.05, 'verbose': False})

# lgbm1 = LGBMClassifier(** {'colsample_bytree': 0.4, 'learning_rate': 0.1, 'max_depth': 5, 
#                           'min_child_samples': 10, 'num_leaves': 40, 'reg_alpha': 0.2, 
#                           'reg_lambda': 0.65, 'subsample': 0.65})

lgbm = LGBMClassifier(**{
                                   'verbose'           : -1,
    
                                   'boosting_type'     : 'gbdt', #'dart',  #gbdt
                                   'random_state'      : 42,
                                   'colsample_bytree'  : 0.4,
                                   'learning_rate'     : 0.10,
                                   'max_depth'         : 3,
                                   'min_child_samples' : 5,
                                   'n_estimators'      : 150,
                                   'num_leaves'        : 40,
                                   'reg_alpha'         : 0.0001,
                                   'reg_lambda'        : 0.65,
                                   'subsample'         : 0.65, 
    
    
                                  }
                      )

# lr2 = LogisticRegression(C=0.1,penalty = 'l2' ,  tol=0.0001, verbose=0, multi_class='ovr',
#                                                    solver = 'lbfgs',max_iter=5000,n_jobs=-1)

lr = LogisticRegression(C=0.1,penalty = 'l2' , multi_class='ovr', tol=0.0001, verbose=0,
                                                   solver = 'lbfgs',max_iter=5000)

# lr1 = LogisticRegression(**{'C': 0.5, 'class_weight': 'balanced', 'max_iter': 300, 
#                            'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'liblinear'})

# svc1=SVC(probability=True,gamma='auto')
svc=SVC(**{'probability':True,'C': 0.01, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1})

# svc2 = SVC(**{'probability':True,'C': 15, 'class_weight': 'balanced', 'decision_function_shape': 'ovo', 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1})

# {'C': 10, 'class_weight': 'balanced', 'decision_function_shape': 'ovo', 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': -1}
# estimators1 = [('xgb',xgb),('lgbm',lgbm1),('cb',cb),('lr',lr),('svc',svc)]

# estimators = [('xgb',xgb),('lgbm',lgbm1),('cb',cb),('lr',lr)]


 

In [27]:
class Ensemble2():
    def __init__(self):
        self.imputer = SimpleImputer(missing_values=np.nan, strategy='median')
        self.sc = StandardScaler()
        self.classifiers =[xgb,lgbm,cb,lr] 
         
    
    def fit(self,X,y): 
        for classifier in self.classifiers:
            if classifier!=self.classifiers[0]:
                X1 = self.sc.fit_transform(X)                
                classifier.fit(X1,y)
            else :
                classifier.fit(X, y)
     
    def predict_proba(self, x):
        probabilities = {}
        i=1
        for classifier in self.classifiers:
            if classifier!=self.classifiers[0]:
                x1 = self.sc.transform(x)
                prob = classifier.predict_proba(x1)
            else:
                prob = classifier.predict_proba(x)
            probabilities[i]=prob
            i+=1
#             [0.01796878307878434, 0.33245804284730207, 0.5462479882932675, 0.1033251857806461]
#             [0.000631933404525617, 0.3766361624722337, 0.5459251005613784, 0.0768068035618623]
#             [0.0004391345036092083, 0.31393309707096045, 0.6190759666796409, 0.06655180174578942]

# [0.8093801703891068, 0.1205553028031325, 0.007347628413365422, 0.06271689839439532]
# [0.7668911687543616, 0.14315443167281441, 0.0004410324469045659, 0.08951336712591937]
# [0.903403487586182, 0.09568024892816951, 0.00036780456678608037, 0.000548458918862374]
# [0.8821638424887391, 0.02306088170170961, 0.00013519731131200361, 0.0946400784982394]
#         averaged_probabilities = probabilities[1]*0.90338 + probabilities[2]*0.0957
#         + probabilities[3]*0.00036  + probabilities[4]*0.00055 #+ probabilities[4]*0.1
        
#         averaged_probabilities = probabilities[1]*0.88215 + probabilities[2]*0.0231
#         + probabilities[3]*0.00015  + probabilities[4]*0.0946 #+ probabilities[4]*0.1
        
        
        averaged_probabilities = probabilities[1]*0.159 + probabilities[2]*0.244
        + probabilities[3]*0.4831 + probabilities[4]*0.1139 #+ probabilities[4]*0.1
        
#         probabilities = np.stack([classifier.predict_proba(x) for classifier in self.classifiers])
#         averaged_probabilities = np.mean(probabilities, axis=0)
        class_0_est_instances = averaged_probabilities[:, 0].sum()
        others_est_instances = averaged_probabilities[:, 1:].sum()
        # Weighted probabilities based on class imbalance
        new_probabilities = averaged_probabilities * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(averaged_probabilities.shape[1])]])
        return new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1) 

In [28]:
class Ensemble3():
    def __init__(self):
        self.imputer = SimpleImputer(missing_values=np.nan, strategy='median')
        self.sc = StandardScaler()
        self.classifiers =[xgb,lgbm,cb,lr] 
         
    
    def fit(self,X,y): 
        for classifier in self.classifiers:
            if classifier!=self.classifiers[0]:
                X1 = self.sc.fit_transform(X)                
                classifier.fit(X1,y)
            else :
                classifier.fit(X, y)
     
    def predict_proba(self, x):
        probabilities = {}
        i=1
        for classifier in self.classifiers:
            if classifier!=self.classifiers[0]:
                x1 = self.sc.transform(x)
                prob = classifier.predict_proba(x1)
            else:
                prob = classifier.predict_proba(x)
            probabilities[i]=prob
            i+=1
 
        
        
        averaged_probabilities = probabilities[1]*0.159 + probabilities[2]*0.244
        + probabilities[3]*0.4831 + probabilities[4]*0.1139 #+ probabilities[4]*0.1
 
    
        return averaged_probabilities

In [29]:
# clf1=CalibratedClassifierCV(xgb,n_jobs=-1,cv=5,method='sigmoid')
    
# clf2=CalibratedClassifierCV(lgbm1,n_jobs=-1,cv=5,method='sigmoid')

# clf3=CalibratedClassifierCV(LogisticRegression(C=0.1,penalty = 'l2' , multi_class='ovr', tol=0.0001, verbose=0,
#                                                solver = 'lbfgs',max_iter=5000) ,n_jobs=-1,
#                             cv=5,method='sigmoid')

# clf4=CalibratedClassifierCV(cb ,n_jobs=-1,cv=5,method='sigmoid') 
# clf5=CalibratedClassifierCV(svc ,n_jobs=-1,cv=5,method='sigmoid') 

# estimators1 = [('xgb',clf1),('lgbm',clf2),('cb',clf4),('lr',clf3)]

In [30]:
# def train_lr(x,y_meta):     
    
#     le = LabelEncoder()
#     y_meta1 = le.fit_transform(y_meta)
#     print(le.classes_)      
    
#     impute = SimpleImputer(missing_values=np.nan, strategy='median')
#     x1 = impute.fit_transform(x)  
 
#     clf1=CalibratedClassifierCV(xgb,n_jobs=-1,cv=5,method='sigmoid')
    
#     clf2=CalibratedClassifierCV(lgbm,n_jobs=-1,cv=5,method='sigmoid')
    
#     clf3=CalibratedClassifierCV(LogisticRegression(C=0.1,penalty = 'l2' , multi_class='ovr', tol=0.0001, verbose=0,
#                                                    solver = 'lbfgs',max_iter=5000) ,n_jobs=-1,
#                                 cv=5,method='sigmoid')
    
#     clf4=CalibratedClassifierCV(cb ,n_jobs=-1,cv=5,method='sigmoid') 
#     clf5=CalibratedClassifierCV(svc ,n_jobs=-1,cv=5,method='sigmoid') 
    
    
#     sc=StandardScaler()
#     x2=sc.fit_transform(x1)
    
#     clf1.fit(x2,y_meta1)
#     print("xgboost classifier fitted")
    
#     clf2.fit(x2,y_meta)
#     print("lgbm classifier fitted")
    
#     clf3.fit(x2,y_meta)
#     print("logistic regression fitted")
    
#     clf4.fit(x2,y_meta)
#     print("catboost fitted")
    
#     clf5.fit(x1,y_meta)
#     print("SVC fitted")
          
#     return sc,impute,clf1,clf2, clf3, clf4, clf5

In [31]:
from tqdm.notebook import tqdm

In [32]:
def ScoreMetric(ytrue, ypred): 
    
    nc = np.bincount(ytrue);
    return log_loss(ytrue, ypred, sample_weight = 1 / nc[ytrue], eps=1e-15)

In [33]:
# import optuna 

# def objective(trial,classifiers,x_val,y_val):
#     # Define the weights of the ensemble as trial parameters (uniform distribution between 0 and 1)
     
#     num_classifiers = len(classifiers)
#     weights = [trial.suggest_uniform(f'weight_{i}', 0, 1) for i in range(num_classifiers)]

# #     weights = [trial.suggest_float(f'weight_{i}', 0, 1/num_classifiers) for i in range(num_classifiers-1)]
# #     weights.append(1.0 - sum(weights))  # The last weight is computed to ensure they sum to 1

# #     Normalize the weights to sum up to 1
#     total_weight = sum(weights)
#     weights = [w / total_weight for w in weights]  
     

    
#     # Create the ensemble predictions using the weighted average of each classifier's predictions
#     ensemble_predictions = sum(weight * classifier.predict_proba(x_val) for weight, classifier in zip(weights, classifiers))

#     # Choose the final predicted class based on the maximum probability
# #     y_pred = np.argmax(ensemble_predictions, axis=1)

#     # Calculate the metric to be maximized (e.g., accuracy)
# #     metric = accuracy_score(y_test, y_pred)
#     metric = ScoreMetric(y_val, 1-ensemble_predictions[:,0])

#     return metric


In [34]:
# # %%time
# # Create Optuna study
# study = optuna.create_study(direction='maximize')

# # Optimize the study with a specified number of trials
# num_trials = 100
# study.optimize(objective, n_trials=num_trials)

# # Get the best weights from the Optuna study
# best_weights = [study.best_params[f'weight_{i}'] for i in range(num_classifiers)]


In [35]:
n_splits=10
cv_innerz = KF(n_splits, shuffle=True, random_state=42)
def training1(data,greeks): 
    train_pred = data.copy(deep=True)
    train_pred  = feature(train_pred )     
    
    y  = train_pred.Class
    x  = train_pred.drop(['Class', 'Id'],axis=1) 
    le = LabelEncoder()   
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
     
#     imputer_knn = KNNImputer(n_neighbors=4) #, weights="distance")
    sc=StandardScaler()
    
    outer_results = list()
    best_loss = np.inf
    split = 0
    splits = n_splits
    models = {}
    for train_idx,val_idx in tqdm(cv_innerz.split(x), total = splits):
        split+=1
        x_train, x_val = x.iloc[train_idx],x.iloc[val_idx]
        y_train, y_val = greeks.Alpha.iloc[train_idx], y.iloc[val_idx]
        y_meta1 = le.fit_transform(y_train)  
        
        ros = RandomOverSampler(random_state=42)
        x_train1,y_train1 = ros.fit_resample(x_train, y_meta1) 
        
        x_train1 = imputer.fit_transform(x_train1)
        x_val = imputer.transform(x_val)
        
#         x_train1 = sc.fit_transform(x_train1)
#         x_val = sc.transform(x_val)
        
        model = Ensemble2()
#         model = cb
#         params = {'alpha': 0.0001, 'colsample_bytree': 0.7, 'lambda': 0.01, 'learning_rate': 0.2, 'max_depth': 20, 'min_child_weight': 10, 'subsample': 0.8}

#         model = xgboost.XGBClassifier(**params)
        
#         model = VotingClassifier(estimators1, voting='soft', weights=[0.3,0.3,0.3,0.1], n_jobs=-1, flatten_transform=True, verbose=False)
#         model = xgb
#         model = TabPFNClassifier(N_ensemble_configurations=24)
        model.fit(x_train1, y_train1) #,overwrite_warning=True)
        models[split]=model
        y_pred = model.predict_proba(x_val)
        probabilities = np.concatenate((y_pred[:,:1], np.sum(y_pred[:,1:], 1, keepdims=True)), axis=1)
        p0 = probabilities[:,0]
        
        
         
#         y_p = np.empty((y_pred.shape[0],))
#         for i in range(y_pred.shape[0]):
#             if p0[i]>=0.5:
#                 y_p[i]= False
#             else :
#                 y_p[i]=True
#         y_p = y_p.astype(int)
        
        loss1 = balanced_log_loss(y_val,1-p0)
        loss = ScoreMetric(y_val,1-p0)
        print("balanced log loss ",loss1)
        print("ScoreMetric ",loss)

        if loss<best_loss:
            best_model = model
            best_loss = loss
            print('best_model_saved')             
            
        outer_results.append(loss)
        print('>val_loss=%.5f, split = %.1f' % (loss,split))
    print('LOSS: %.5f' % (np.mean(outer_results)))
    return best_model, models 

train_pred_and_time = get_traindf()
m,models = training1(train_pred_and_time,greeks)
    
#xgb - 0.30
#lgbm1 - 0.35
#cb - 0.30
#lr = 0.48
#svc1 - 0.86
#svc - 0.78
#tabpfn24 - 0.35
#64- 0.38

#[0.3,0.2,0.2,0.1,0.1,0.1] - 

#weights=[0.3,0.2,0.3,0.1,0.1] - 0.27

#ensemble -eq weights - 0.33564
#VC - eq weights - 0.29797
#VC with 4 -0.27179
#ensemble with 4 - equ weights - cv10 - 0.22978
#vc with 4 - cv10 - 0.3/0.3/0.3/0.1 - 0.2512 --> for ensemble - 0.20255 cv5 -0.21695

# LOSS: 0.23912 for sigmoid - 4
# LOSS: 0.32263 for sigmoid estimators2 (without weighting for imbalancce)
#with one tabpfn - ensemble2 - 5 - LOSS: 0.21427
#with two tabpfn - ensemble2 - 5 - LOSS: 0.210 bm-val_loss=0.15953

#without lr - LOSS: 0.21751  -  with lr - 0.21427  with svcinstead - 0.21427?

#cb with classwt - 0.30462


# best weights [0.394966529199084, 0.10961102417573335, 0.00149112820778172, 0.4939313184174009]

#LOSS: 0.19882
#LOSS: 0.19785

#cb 0.28882 vs 0.28417 vs 0.25303

#LOSS: 0.20914

# LOSS: 0.19215

  0%|          | 0/10 [00:00<?, ?it/s]

balanced log loss  0.10320533436415286
ScoreMetric  0.15569909940965607
best_model_saved
>val_loss=0.15570, split = 1.0
balanced log loss  0.07796397471270332
ScoreMetric  0.14408342249789982
best_model_saved
>val_loss=0.14408, split = 2.0
balanced log loss  0.05449055870376578
ScoreMetric  0.07427720129690628
best_model_saved
>val_loss=0.07428, split = 3.0
balanced log loss  0.15549481237269386
ScoreMetric  0.26636455381489993
>val_loss=0.26636, split = 4.0
balanced log loss  0.050066473906860164
ScoreMetric  0.12497112058309776
>val_loss=0.12497, split = 5.0
balanced log loss  0.10064425928711389
ScoreMetric  0.20276547835412256
>val_loss=0.20277, split = 6.0
balanced log loss  0.34015307529031413
ScoreMetric  0.4864391448720119
>val_loss=0.48644, split = 7.0
balanced log loss  0.07791802097304944
ScoreMetric  0.17095103540136614
>val_loss=0.17095, split = 8.0
balanced log loss  0.08255780599730973
ScoreMetric  0.1306112228384309
>val_loss=0.13061, split = 9.0
balanced log loss  0.08

In [36]:
# cv_innerz = KF(n_splits = 5, shuffle=True, random_state=42)
# def training4(data,greeks): 
#     train_pred = data.copy(deep=True)
#     train_pred  = feature(train_pred )
    
# #     classes_weights = class_weight.compute_sample_weight(
# #     class_weight='balanced',
# #     y=train_pred['Class']) 
    
    
#     y  = train_pred.Class
#     x  = train_pred.drop(['Class', 'Id'],axis=1)  
 
    
#     le = LabelEncoder()   
#     imputer = SimpleImputer(missing_values=np.nan, strategy='median')
     
# #     imputer_knn = KNNImputer(n_neighbors=4) #, weights="distance")
#     sc=StandardScaler()
     
    
#     outer_results = list()
#     best_loss = np.inf
#     split = 0
#     splits = 5
#     models = {}
#     for train_idx,val_idx in tqdm(cv_innerz.split(x), total = splits):
#         split+=1
#         x_train, x_val = x.iloc[train_idx],x.iloc[val_idx]
#         y_train, y_val = greeks.Alpha.iloc[train_idx], y.iloc[val_idx]
#         y_meta1 = le.fit_transform(y_train)       
     
        
#         rus = RandomUnderSampler(random_state=42)
#         x_train1,y_train1 = rus.fit_resample(x_train, y_meta1) 
        
#         x_train1 = imputer.fit_transform(x_train1)
        
#         x_train1 = sc.fit_transform(x_train1)
         
#         x_val = imputer.transform(x_val)
#         x_val = sc.transform(x_val)
        
        
        
#         model = Ensemble2()
# #         model = lgbm
# #         params = {'alpha': 0.0001, 'colsample_bytree': 0.7, 'lambda': 0.01, 'learning_rate': 0.2, 'max_depth': 20, 'min_child_weight': 10, 'subsample': 0.8}

# #         model = xgboost.XGBClassifier(**params)
        
# #         model = VotingClassifier(estimators1, voting='soft', weights=[0.3,0.3,0.3,0.1], n_jobs=-1, flatten_transform=True, verbose=False)
# #         model = xgb
# #         model = TabPFNClassifier(N_ensemble_configurations=24)
#         model.fit(x_train1, y_train1) #,overwrite_warning=True)
#         models[split]=model
#         y_pred = model.predict_proba(x_val)
#         probabilities = np.concatenate((y_pred[:,:1], np.sum(y_pred[:,1:], 1, keepdims=True)), axis=1)
#         p0 = probabilities[:,0]
        
        
         
# #         y_p = np.empty((y_pred.shape[0],))
# #         for i in range(y_pred.shape[0]):
# #             if p0[i]>=0.5:
# #                 y_p[i]= False
# #             else :
# #                 y_p[i]=True
# #         y_p = y_p.astype(int)
        
#         loss1 = balanced_log_loss(y_val,1-p0)
#         loss = ScoreMetric(y_val,1-p0)
#         print("balanced log loss ",loss1)
#         print("ScoreMetric ",loss)

#         if loss<best_loss:
#             best_model = model
#             best_loss = loss
#             print('best_model_saved')
#         outer_results.append(loss)
#         print('>val_loss=%.5f, split = %.1f' % (loss,split))
#     print('LOSS: %.5f' % (np.mean(outer_results)))
#     return best_model, models, sc, imputer

# m,models,sc,imputer = training4(train_pred_and_time,greeks)   


In [37]:
# %%time
# cv_innerz = KF(n_splits = 10, shuffle=True, random_state=42)
# def trainingz(data,greeks): 
#     train_pred = data.copy(deep=True)
#     train_pred  = feature(train_pred ) 
    
    
#     y  = train_pred.Class
#     x  = train_pred.drop(['Class', 'Id'],axis=1)
    
 
    
#     le = LabelEncoder()
    
#     impute = SimpleImputer(missing_values=np.nan, strategy='median') 
#     sc=StandardScaler()
     
    
#     outer_results = list()
#     best_loss = np.inf
#     split = 0
#     splits = 10
#     models = {}
#     for train_idx,val_idx in tqdm(cv_innerz.split(x), total = splits):
#         split+=1
#         x_train, x_val = x.iloc[train_idx],x.iloc[val_idx]
#         y_train, y_val = greeks.Alpha.iloc[train_idx], y.iloc[val_idx]
#         y_meta1 = le.fit_transform(y_train)       
     
        
#         ros = RandomOverSampler(random_state=42)
#         x_train1,y_train1 = ros.fit_resample(x_train, y_meta1)   
#         x_train1= impute.fit_transform(x_train1) 
# #         x_train1 = sc.fit_transform(x_train1)
#         x_val = impute.transform(x_val)
# #         x_val = sc.transform(x_val)
        
        
        
#         model = Ensemble2()
# #         model = cb
# #         params = {'alpha': 0.0001, 'colsample_bytree': 0.7, 'lambda': 0.01, 'learning_rate': 0.2, 'max_depth': 20, 'min_child_weight': 10, 'subsample': 0.8}

# #         model = xgboost.XGBClassifier(**params)
        
# #         model = VotingClassifier(estimators1, voting='soft', weights=[0.3,0.3,0.3,0.1], n_jobs=-1, flatten_transform=True, verbose=False)
# #         model = xgb
# #         model = TabPFNClassifier(N_ensemble_configurations=24)
#         model.fit(x_train1, y_train1) #,overwrite_warning=True)
#         models[split]=model
#         y_pred = model.predict_proba(x_val)
#         probabilities = np.concatenate((y_pred[:,:1], np.sum(y_pred[:,1:], 1, keepdims=True)), axis=1)
#         p0 = probabilities[:,0]
        
#         # Create Optuna study
        

#         # Set the logging level to silent (no outputs)
# #         optuna.logging.set_verbosity(optuna.logging.INFO)
#         optuna.logging.set_verbosity(optuna.logging.WARNING)

#         study = optuna.create_study(direction='minimize')

#         # Optimize the study with a specified number of trials
#         num_trials = 200
#         print("optuna study starts")
#         classifiers = [xgb,lgbm,cb,lr]
#         num_classifiers = len(classifiers)
# #         partial_objective = partial(objective,classifiers,x_val,y_val)
#         study.optimize(lambda trial: objective(trial, classifiers,x_val,y_val),n_trials=num_trials)
# #         study.optimize(partial_objective, n_trials=num_trials)
         
#         print("optuna study completed")

#         # Get the best weights from the Optuna study
#         best_weights = [study.best_params[f'weight_{i}'] for i in range(num_classifiers)]
        
#         total_weight = sum(best_weights)
#         best_weights_normalized = [w / total_weight for w in best_weights]
#         print("best weights",best_weights_normalized)
#         print("best score of metric ",study.best_value) 
        
#         loss1 = balanced_log_loss(y_val,1-p0)
#         loss = ScoreMetric(y_val,1-p0)
#         print("balanced log loss ",loss1)
#         print("ScoreMetric ",loss)

#         if loss<best_loss:
#             best_model = model
#             best_loss = loss
#             print('best_model_saved')
#         outer_results.append(loss)
#         print('>val_loss=%.5f, split = %.1f' % (loss,split))
#     print('LOSS: %.5f' % (np.mean(outer_results)))
#     return best_model, models, sc, impute

# m,models,sc,impute = trainingz(train_pred_and_time,greeks)
    
# #xgb - 0.30
# #lgbm1 - 0.35
# #cb - 0.30
# #lr = 0.48
# #svc1 - 0.86
# #svc - 0.78
# #tabpfn24 - 0.35
# #64- 0.38

# #[0.3,0.2,0.2,0.1,0.1,0.1] - 

# #weights=[0.3,0.2,0.3,0.1,0.1] - 0.27

# #ensemble -eq weights - 0.33564
# #VC - eq weights - 0.29797
# #VC with 4 -0.27179
# #ensemble with 4 - equ weights - cv10 - 0.22978
# #vc with 4 - cv10 - 0.3/0.3/0.3/0.1 - 0.2512 --> for ensemble - 0.20255 cv5 -0.21695

# # LOSS: 0.23912 for sigmoid - 4
# # LOSS: 0.32263 for sigmoid estimators2 (without weighting for imbalancce)
# #with one tabpfn - ensemble2 - 5 - LOSS: 0.21427
# #with two tabpfn - ensemble2 - 5 - LOSS: 0.210 bm-val_loss=0.15953

# #without lr - LOSS: 0.21751  -  with lr - 0.21427  with svcinstead - 0.21427?

# #cb with classwt - 0.30462


# # best weights [0.394966529199084, 0.10961102417573335, 0.00149112820778172, 0.4939313184174009]

In [38]:
cv_inner = KF(n_splits = 5, shuffle=True, random_state=42)
def training2(data,y,y_meta):
    x = data.copy(deep=True)
    models={}
    outer_results = list()
    best_loss = np.inf
    split = 0
    splits = 5
    for train_idx,val_idx in tqdm(cv_inner.split(x), total = splits):
        split+=1
        x_train, x_val = x.iloc[train_idx],x.iloc[val_idx]
        y_train, y_val = y_meta.iloc[train_idx], y.iloc[val_idx]
        model = Ensemble()
        model.fit(x_train, y_train)
        models[split]=model
        y_pred = model.predict_proba(x_val)
        probabilities = np.concatenate((y_pred[:,:1], np.sum(y_pred[:,1:], 1, keepdims=True)), axis=1)
        p0 = probabilities[:,:1]
         
        y_p = np.empty((y_pred.shape[0],))
        for i in range(y_pred.shape[0]):
            if p0[i]>=0.5:
                y_p[i]= False
            else :
                y_p[i]=True
        y_p = y_p.astype(int)
        loss = balanced_log_loss(y_val,y_p)
        loss1 = ScoreMetric(y_val,y_p)
        print("Score metric ",loss1)
        print("Balanced logloss ",loss)

        if loss<best_loss:
            best_model = model
            best_loss = loss
            xtrain = x_train
            xval = x_val
            ytrain = y_train
            yval = y_val
            print('best_model_saved')
        outer_results.append(loss)
        print('>val_loss=%.5f, split = %.1f' % (loss,split))
    print('LOSS: %.5f' % (np.mean(outer_results)))
    return best_model, models, xtrain,xval,ytrain,yval

In [39]:
from datetime import datetime
times = greeks.Epsilon.copy()
times[greeks.Epsilon != 'Unknown'] = greeks.Epsilon[greeks.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
times[greeks.Epsilon == 'Unknown'] = np.nan

In [40]:
predictor_columns = [n for n in train.columns if n != 'Class' and n != 'Id']

In [41]:
# train  = feature(train )
# test = feature(test)
# predictor_columns = predictor_columns + ['out_GL']
 

In [42]:
train_pred_and_time = pd.concat((train, times), axis=1)
# test_predictors = test[predictor_columns]
# first_category = test_predictors.EJ.unique()[0]
# test_predictors.EJ = test_predictors.EJ.eq(first_category).astype('int')
# test_pred_and_time = np.concatenate((test_predictors, np.zeros((len(test_predictors), 1)) + train_pred_and_time.Epsilon.max() + 1), axis=1)

In [43]:
# cols = train_pred_and_time.columns.tolist()
# cols

In [44]:
# cols.remove('BQ')
# cols.remove('EL')

In [45]:
# cols.remove('Id')
# cols.remove('Class')

In [46]:
ros = RandomOverSampler(random_state=42)

train_ros, y_ros = ros.fit_resample(train_pred_and_time, greeks.Alpha)
print('Original dataset shape')
print(greeks.Alpha.value_counts())
print('Resample dataset shape')
print( y_ros.value_counts())

Original dataset shape
A    509
B     61
G     29
D     18
Name: Alpha, dtype: int64
Resample dataset shape
B    509
A    509
D    509
G    509
Name: Alpha, dtype: int64


In [47]:
x_ros = train_ros.drop(['Class', 'Id'],axis=1)
y_ = train_ros.Class

In [48]:
# ypred = stacking_classifier.predict_proba(x_test)
# y_test1 = np.where(y_test == 0, 0, 1)
# score = ScoreMetric(y_test1,1-ypred[:,0])
# score

In [49]:
%%time
best_model2, models2,xtrain,xval,ytrain,yval= training2(x_ros,y_,y_ros)

  0%|          | 0/5 [00:00<?, ?it/s]

Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Score metric  0.1541945356800942
Balanced logloss  0.12283393999583053
best_model_saved
>val_loss=0.12283, split = 1.0
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


/tmp/ipykernel_20/2151898688.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.EJ = X.EJ.eq(first_category).astype('int')


Score metric  9.992007221626417e-16
Balanced logloss  7.882664572210757e-16
best_model_saved
>val_loss=0.00000, split = 2.0
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


/tmp/ipykernel_20/2151898688.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.EJ = X.EJ.eq(first_category).astype('int')


Score metric  9.992007221626413e-16
Balanced logloss  7.927542919637485e-16
>val_loss=0.00000, split = 3.0
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


/tmp/ipykernel_20/2151898688.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.EJ = X.EJ.eq(first_category).astype('int')


Score metric  9.992007221626413e-16
Balanced logloss  6.883759419809394e-16
best_model_saved
>val_loss=0.00000, split = 4.0
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters


/tmp/ipykernel_20/2151898688.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.EJ = X.EJ.eq(first_category).astype('int')


Score metric  0.20081148832756426
Balanced logloss  0.13386381920254847
>val_loss=0.13386, split = 5.0
LOSS: 0.05134
CPU times: user 33min 15s, sys: 12min 59s, total: 46min 15s
Wall time: 23min 50s


In [50]:
y_.value_counts()/y_.shape[0]

1    0.75
0    0.25
Name: Class, dtype: float64

In [51]:
train_pred_and_time = pd.concat((train, times), axis=1)
test_predictors = test[predictor_columns]
first_category = test_predictors.EJ.unique()[0]
test_predictors.EJ = test_predictors.EJ.eq(first_category).astype('int')
# test_pred_and_time = np.concatenate((test_predictors, np.zeros((len(test_predictors), 1)) + train_pred_and_time.Epsilon.max() + 1), axis=1)

/tmp/ipykernel_20/1202244907.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_predictors.EJ = test_predictors.EJ.eq(first_category).astype('int')


In [52]:
test_predictors['Epsilon']=train_pred_and_time.Epsilon.max() + 1

In [53]:
test_predictors.columns

Index(['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN', 'BP',
       'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS', 'CU',
       'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY', 'EB',
       'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL',
       'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL', 'Epsilon'],
      dtype='object')

In [54]:
%%time
y_pred2={}
# for index,m in enumerate(models.keys()):
for index in range(5):
    y_pred2[index+1] = models2[index+1].predict_proba(test_predictors)
    
ypred_model2_final = (y_pred2[1]+y_pred2[2]+y_pred2[3]+y_pred2[4]+y_pred2[5])/5

# y_best2 = best_model2.predict_proba(test_predictors)

ypred_model2_final

CPU times: user 26min 5s, sys: 8min 55s, total: 35min
Wall time: 17min 57s


array([[0.49999999, 0.25577551, 0.11967627, 0.12454824],
       [0.49999999, 0.25577551, 0.11967627, 0.12454824],
       [0.49999999, 0.25577551, 0.11967627, 0.12454824],
       [0.49999999, 0.25577551, 0.11967627, 0.12454824],
       [0.49999999, 0.2557755 , 0.11967627, 0.12454824]])

In [55]:
def get_testpredictors():
    
    train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
    test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
    greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
    times = greeks.Epsilon.copy()
    times[greeks.Epsilon != 'Unknown'] = greeks.Epsilon[greeks.Epsilon != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())
    times[greeks.Epsilon == 'Unknown'] = np.nan

    predictor_columns = [n for n in train.columns if n != 'Class' and n != 'Id']

    train_pred_and_time = pd.concat((train, times), axis=1)
    test_predictors = test[predictor_columns]
    
    first_category = test_predictors.EJ.unique()[0]
    test_predictors.EJ = test_predictors.EJ.eq(first_category).astype('int')
    test_predictors['Correction'] =0

    test_predictors['Epsilon']=train_pred_and_time.Epsilon.max() + 1
    
    
    test_predictors = feature(test_predictors)
    
    return test_predictors



In [56]:
n_splits=10
cv_innerz = KF(n_splits, shuffle=True, random_state=42)
ypred_model1 =[]
def model1_predict(data,greeks,test_predictors): 
    train_pred = data.copy(deep=True)
    train_pred  = feature(train_pred )  
#     test_predictors = feature(test_predictors)
    
    y  = train_pred.Class
    x  = train_pred.drop(['Class', 'Id'],axis=1) 
    le = LabelEncoder()   
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
     
#     imputer_knn = KNNImputer(n_neighbors=4) #, weights="distance")
    sc=StandardScaler()
    
    outer_results = list()
    best_loss = np.inf
    split = 0
    splits = n_splits
    models = {}
    for train_idx,val_idx in tqdm(cv_innerz.split(x), total = splits):
        split+=1
        x_train, _ = x.iloc[train_idx],x.iloc[val_idx]
        y_train, _ = greeks.Alpha.iloc[train_idx], y.iloc[val_idx]
        y_meta1 = le.fit_transform(y_train)  
        
        x_val = test_predictors.copy(deep=True)
        
        ros = RandomOverSampler(random_state=42)
        x_train1,y_train1 = ros.fit_resample(x_train, y_meta1) 
        
        x_train1 = imputer.fit_transform(x_train1)
        x_val = imputer.transform(x_val)
        
#         x_train1 = sc.fit_transform(x_train1)
#         x_val = sc.transform(x_val)
        
        model = Ensemble3()
#         model = cb
#         params = {'alpha': 0.0001, 'colsample_bytree': 0.7, 'lambda': 0.01, 'learning_rate': 0.2, 'max_depth': 20, 'min_child_weight': 10, 'subsample': 0.8}

#         model = xgboost.XGBClassifier(**params)
        
#         model = VotingClassifier(estimators1, voting='soft', weights=[0.3,0.3,0.3,0.1], n_jobs=-1, flatten_transform=True, verbose=False)
#         model = xgb
#         model = TabPFNClassifier(N_ensemble_configurations=24)
        model.fit(x_train1, y_train1) #,overwrite_warning=True)
        models[split]=model
        y_pred = model.predict_proba(x_val)
#         probabilities = np.concatenate((y_pred[:,:1], np.sum(y_pred[:,1:], 1, keepdims=True)), axis=1)
        
        ypred_model1.append(y_pred)
#         p0 = probabilities[:,0]
        
        
         
#         y_p = np.empty((y_pred.shape[0],))
#         for i in range(y_pred.shape[0]):
#             if p0[i]>=0.5:
#                 y_p[i]= False
#             else :
#                 y_p[i]=True
#         y_p = y_p.astype(int)
        
#         loss1 = balanced_log_loss(y_val,1-p0)
#         loss = ScoreMetric(y_val,1-p0)
#         print("balanced log loss ",loss1)
#         print("ScoreMetric ",loss)

#         if loss<best_loss:
#             best_model = model
#             best_loss = loss
#             print('best_model_saved')             
            
#         outer_results.append(loss)
#         print('>val_loss=%.5f, split = %.1f' % (loss,split))
#     print('LOSS: %.5f' % (np.mean(outer_results)))

    ypred_model1_final = np.stack(ypred_model1)
#     ypred_stack_final = np.concatenate(ypred_stack,axis=0)
    ypred_model1_final = np.mean(ypred_model1_final,axis=0)
    
    class_0_est_instances = ypred_model1_final[:, 0].sum()
    others_est_instances = ypred_model1_final[:, 1:].sum()
    # Weighted probabilities based on class imbalance
    new_probabilities = ypred_model1_final * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(ypred_model1_final.shape[1])]])
    ypred_model1_final = new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1) 
    return ypred_model1_final



In [57]:
%%time 
test_predictors = get_testpredictors()
train_data = get_traindf()

ypred_model1_final = model1_predict(train_data,greeks,test_predictors)

/tmp/ipykernel_20/2408710838.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_predictors.EJ = test_predictors.EJ.eq(first_category).astype('int')
/tmp/ipykernel_20/3236182820.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.loc[X['GL']<1,'out_GL'] = X.loc[X['GL']<1,'GL'].map(lambda x : x-X.loc[X['GL']<1,'GL'].mean())
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating

  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 3min 50s, sys: 50.8 s, total: 4min 40s
Wall time: 1min 21s


In [58]:
ypred_model1_final

array([[0.5       , 0.07201761, 0.23380481, 0.19417758],
       [0.5       , 0.07201761, 0.23380481, 0.19417758],
       [0.5       , 0.07201761, 0.23380481, 0.19417758],
       [0.5       , 0.07201761, 0.23380481, 0.19417758],
       [0.5       , 0.07201761, 0.23380481, 0.19417758]])

In [59]:
n_splits = 10
cv_innerz = KF(n_splits, shuffle=True, random_state=42)
ypred_stack = []
def train_stack(data,greeks,test_predictors): 
    train_pred = data.copy(deep=True)
    train_pred  = feature(train_pred )   
#     test_predictors = feature1(test_predictors)
     
    y  = train_pred.Class
    x  = train_pred.drop(['Class', 'Id'],axis=1) 
    le = LabelEncoder()   
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
     
#     imputer_knn = KNNImputer(n_neighbors=4) #, weights="distance")
    sc=StandardScaler()
    
    outer_results = list()
    best_loss = np.inf
    split = 0
    splits = n_splits
    models = {}
    for train_idx,val_idx in tqdm(cv_innerz.split(x), total = splits):
#         x=shuffle(x)
        split+=1
        x_train, x_val = x.iloc[train_idx],x.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
#         y_meta1 = le.fit_transform(y_train)   
        
        ros = RandomOverSampler(random_state=42)
        x_train1,y_train1 = ros.fit_resample(x_train, y_train) 
        x_train1 = imputer.fit_transform(x_train1)
        x_train1 = sc.fit_transform(x_train1)
#         x_val = imputer.transform(test_predictors)
        x_val = imputer.transform(x_val)
        x_val = sc.transform(x_val) 
        estimators = [ ('lgbm',lgbm),('cb',cb),('lr',lr),('svc',svc)]
         
        model = StackingClassifier(estimators=estimators, cv=5,stack_method = 'predict_proba',
                                         final_estimator=LogisticRegression(),
                                         passthrough = False,verbose=0)
        
        model.fit(x_train1, y_train1) #,overwrite_warning=True)
        models[split]=model
        probabilities = model.predict_proba(x_val)    
        
        class_0_est_instances = probabilities[:, 0].sum()
        others_est_instances = probabilities[:, 1:].sum()
        # Weighted probabilities based on class imbalance
        new_probabilities = probabilities * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(probabilities.shape[1])]])
        probabilities = new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1) 
        
#         ypred_stack.append(probabilities)
        p0 = probabilities[:,0] 
    
#         p0[p0>0.25] = 1
#         p0[p0<0.01] = 0
        
        loss1 = balanced_log_loss(y_val,1-p0) 
        loss = ScoreMetric(y_val,1-p0)
        
        print("balanced log loss ",loss1)
        print("ScoreMetric ",loss)

        if loss<best_loss:
            best_model = model
            best_loss = loss
            print('best_model_saved')             
            
        outer_results.append(loss)
        print('>val_loss=%.5f, split = %.1f' % (loss,split))

#     ypred_stack_final = np.stack(ypred_stack)
# #     ypred_stack_final = np.concatenate(ypred_stack,axis=0)
#     ypred_stack_final = np.mean(ypred_stack_final,axis=0)
    
#     class_0_est_instances = ypred_stack_final[:, 0].sum()
#     others_est_instances = ypred_stack_final[:, 1:].sum()
#     # Weighted probabilities based on class imbalance
#     new_probabilities = ypred_stack_final * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(ypred_stack_final.shape[1])]])
#     ypred_stack_final1 = new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1) 
        
    print("ScoreMetric ",ScoreMetric(y_val,1-p0))
    print('LOSS: %.5f' % (np.mean(outer_results)))
#     return models #ypred_stack_final1
    return probabilities

 
test_predictors1 = get_testpredictors()

models = train_stack(get_traindf(),greeks, test_predictors1)  
# models
 
#LOSS: 0.21651 -lgb,cb,lr
#LOSS: 0.21650 withsvc
#LOSS: 0.21316 with sc/svc
#LOSS: 0.21773  - with sc LOSS: 0.21902 for feataure1
#LOSS: 0.21897 -withsc - trndf?
#LOSS: 0.21771 without sc feature1
#LOSS: 0.21651 with feature1 and without sc vx LOSS: 0.21318 fr with sc
#LOSS: 0.18850 for all 1 LOSS: 0.21316 for all 0 #LOSS: 0.21312 for no change
#LOSS: 0.17297 with sc svc and correction feature / feature1

/tmp/ipykernel_20/2408710838.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_predictors.EJ = test_predictors.EJ.eq(first_category).astype('int')
/tmp/ipykernel_20/3236182820.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.loc[X['GL']<1,'out_GL'] = X.loc[X['GL']<1,'GL'].map(lambda x : x-X.loc[X['GL']<1,'GL'].mean())
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating

  0%|          | 0/10 [00:00<?, ?it/s]

balanced log loss  0.1142299149659215
ScoreMetric  0.17233115899882345
best_model_saved
>val_loss=0.17233, split = 1.0
balanced log loss  0.059839174193501585
ScoreMetric  0.11058739692299048
best_model_saved
>val_loss=0.11059, split = 2.0
balanced log loss  0.04092008597363096
ScoreMetric  0.05577901080944592
best_model_saved
>val_loss=0.05578, split = 3.0
balanced log loss  0.10044879191391012
ScoreMetric  0.17207003392026318
>val_loss=0.17207, split = 4.0
balanced log loss  0.08413521931404358
ScoreMetric  0.21001024872934
>val_loss=0.21001, split = 5.0
balanced log loss  0.06234312016696716
ScoreMetric  0.1256011288898437
>val_loss=0.12560, split = 6.0
balanced log loss  0.39911084751825693
ScoreMetric  0.570752268549174
>val_loss=0.57075, split = 7.0
balanced log loss  0.0658785528476425
ScoreMetric  0.14453661270405535
>val_loss=0.14454, split = 8.0
balanced log loss  0.10636863804123511
ScoreMetric  0.16828133594873976
>val_loss=0.16828, split = 9.0
balanced log loss  0.04161718

In [60]:
n_splits = 10
cv_innerz = KF(n_splits, shuffle=True, random_state=42)
ypred_stack = []
def train_stack(data,greeks,test_predictors): 
    train_pred = data.copy(deep=True)
    train_pred  = feature(train_pred )   
#     test_predictors = feature1(test_predictors)
     
    y  = train_pred.Class
    x  = train_pred.drop(['Class', 'Id'],axis=1) 
    le = LabelEncoder()   
    imputer = SimpleImputer(missing_values=np.nan, strategy='median')
     
#     imputer_knn = KNNImputer(n_neighbors=4) #, weights="distance")
    sc=StandardScaler()
    
    outer_results = list()
    best_loss = np.inf
    split = 0
    splits = n_splits
    models = {}
    for train_idx,val_idx in tqdm(cv_innerz.split(x), total = splits):
#         x=shuffle(x)
        split+=1
        x_train, x_val = x.iloc[train_idx],x.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
#         y_meta1 = le.fit_transform(y_train)   
        
        ros = RandomOverSampler(random_state=42)
        x_train1,y_train1 = ros.fit_resample(x_train, y_train) 
        x_train1 = imputer.fit_transform(x_train1)
        x_train1 = sc.fit_transform(x_train1)
#         x_val = imputer.transform(test_predictors)
        x_val = imputer.transform(test_predictors)
        x_val = sc.transform(x_val) 
        estimators = [ ('lgbm',lgbm),('cb',cb),('lr',lr),('svc',svc)]
         
        model = StackingClassifier(estimators=estimators, cv=5,stack_method = 'predict_proba',
                                         final_estimator=LogisticRegression(),
                                         passthrough = False,verbose=0)
        
        model.fit(x_train1, y_train1) #,overwrite_warning=True)
        models[split]=model
        probabilities = model.predict_proba(x_val)    
        
        ypred_stack.append(probabilities)
         

    ypred_stack_final1 = np.stack(ypred_stack) 
    ypred_stack_final2 = np.mean(ypred_stack_final1,axis=0)
    
    class_0_est_instances = ypred_stack_final2[:, 0].sum()
    others_est_instances = ypred_stack_final2[:, 1:].sum()
    # Weighted probabilities based on class imbalance
    new_probabilities = ypred_stack_final2 * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(ypred_stack_final2.shape[1])]])
    ypred_stack_final = new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1)  
    return ypred_stack_final

 
test_predictors1 = get_testpredictors()

ypred_stack_final = train_stack(get_traindf(),greeks, test_predictors1)  
ypred_stack_final
# models
 
#LOSS: 0.21651 -lgb,cb,lr
#LOSS: 0.21650 withsvc
#LOSS: 0.21316 with sc/svc
#LOSS: 0.21773  - with sc LOSS: 0.21902 for feataure1
#LOSS: 0.21897 -withsc - trndf?
#LOSS: 0.21771 without sc feature1
#LOSS: 0.21651 with feature1 and without sc vx LOSS: 0.21318 fr with sc
#LOSS: 0.18850 for all 1 LOSS: 0.21316 for all 0 #LOSS: 0.21312 for no change
#LOSS: 0.17297 with sc svc and correction feature / feature1

/tmp/ipykernel_20/2408710838.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_predictors.EJ = test_predictors.EJ.eq(first_category).astype('int')
/tmp/ipykernel_20/3236182820.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.loc[X['GL']<1,'out_GL'] = X.loc[X['GL']<1,'GL'].map(lambda x : x-X.loc[X['GL']<1,'GL'].mean())
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating

  0%|          | 0/10 [00:00<?, ?it/s]

array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]])

In [61]:

# n_splits = 10
# cv_innerz = KF(n_splits, shuffle=True, random_state=42)
# ypred_stack = []
# def train_stack(data,greeks,test_predictors): 
#     train_pred = data.copy(deep=True)
#     train_pred  = feature(train_pred )   
#     test_predictors = feature(test_predictors)
    
#     y  = train_pred.Class
#     x  = train_pred.drop(['Class', 'Id'],axis=1) 
#     le = LabelEncoder()   
#     imputer = SimpleImputer(missing_values=np.nan, strategy='median')
     
# #     imputer_knn = KNNImputer(n_neighbors=4) #, weights="distance")
#     sc=StandardScaler()
    
#     outer_results = list()
#     best_loss = np.inf
#     split = 0
#     splits = n_splits
#     models = {}
#     for train_idx,val_idx in tqdm(cv_innerz.split(x), total = splits):
# #         x=shuffle(x)
#         split+=1
#         x_train, x_val = x.iloc[train_idx],x.iloc[val_idx]
#         y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
# #         y_meta1 = le.fit_transform(y_train)   
        
#         ros = RandomOverSampler(random_state=42)
#         x_train1,y_train1 = ros.fit_resample(x_train, y_train) 
#         x_train1 = imputer.fit_transform(x_train1)
#         x_train1 = sc.fit_transform(x_train1)
#         x_val = imputer.transform(test_predictors)
# #         x_val = imputer.transform(x_val)
#         x_val = sc.transform(x_val) 
#         estimators = [ ('lgbm',lgbm),('cb',cb),('lr',lr)]
         
#         model = StackingClassifier(estimators=estimators, cv=5,stack_method = 'predict_proba',
#                                          final_estimator=LogisticRegression(),
#                                          passthrough = False,verbose=0)
        
#         model.fit(x_train1, y_train1) #,overwrite_warning=True)
#         models[split]=model
#         probabilities = model.predict_proba(x_val)    
        
# #         class_0_est_instances = probabilities[:, 0].sum()
# #         others_est_instances = probabilities[:, 1:].sum()
# #         # Weighted probabilities based on class imbalance
# #         new_probabilities = probabilities * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(probabilities.shape[1])]])
# #         probabilities = new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1) 
        
#         ypred_stack.append(probabilities)
# #         p0 = probabilities[:,0] 
        
# #         loss1 = balanced_log_loss(y_val,1-p0) 
# #         loss = ScoreMetric(y_val,1-p0)
        
# #         print("balanced log loss ",loss1)
# #         print("ScoreMetric ",loss)

# #         if loss<best_loss:
# #             best_model = model
# #             best_loss = loss
# #             print('best_model_saved')             
            
# #         outer_results.append(loss)
# #         print('>val_loss=%.5f, split = %.1f' % (loss,split))

#     ypred_stack_final = np.stack(ypred_stack)
# #     ypred_stack_final = np.concatenate(ypred_stack,axis=0)
#     ypred_stack_final = np.mean(ypred_stack_final,axis=0)
    
#     class_0_est_instances = ypred_stack_final[:, 0].sum()
#     others_est_instances = ypred_stack_final[:, 1:].sum()
#     # Weighted probabilities based on class imbalance
#     new_probabilities = ypred_stack_final * np.array([[1/(class_0_est_instances if i==0 else others_est_instances) for i in range(ypred_stack_final.shape[1])]])
#     ypred_stack_final1 = new_probabilities / np.sum(new_probabilities, axis=1, keepdims=1) 
        
# #     print("ScoreMetric ",ScoreMetric(y_val,1-ypred_stack_final1))
# #     print('LOSS: %.5f' % (np.mean(outer_results)))
#     return ypred_stack_final1

# # traindf = get_traindf()

# # m,models,sc,imputer = train_stack(traindf,greeks)  
 


In [62]:
# %%time
# test_predictors = get_testpredictors()
# ypred_stack_final = train_stack(train_pred_and_time,greeks, test_predictors)  
# # ypred_stack_final

In [63]:
probabilities1 = np.concatenate((ypred_model1_final[:,:1], np.sum(ypred_model1_final[:,1:], 1, keepdims=True)), axis=1)
probabilities2 = np.concatenate((ypred_model2_final[:,:1], np.sum(ypred_model2_final[:,1:], 1, keepdims=True)), axis=1)

probabilities = 0.3*probabilities1 + 0.4*probabilities2 + 0.3 * ypred_stack_final

# probabilities = 0.5*probabilities1 + 0.5 * ypred_stack_final

p0 = probabilities[:,:1]  

# p0[p0 > 0.8] = 1
# p0[p0 < 0.1] = 0

In [64]:
probabilities1

array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]])

In [65]:
probabilities2

array([[0.49999999, 0.50000001],
       [0.49999999, 0.50000001],
       [0.49999999, 0.50000001],
       [0.49999999, 0.50000001],
       [0.49999999, 0.50000001]])

In [66]:
ypred_stack_final

array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]])

In [67]:
submission = pd.DataFrame(test["Id"], columns=["Id"])
submission["class_0"] = p0
submission["class_1"] = 1 - p0
submission.to_csv('submission.csv', index=False)

In [68]:
submission_df = pd.read_csv('submission.csv')
submission_df

,Id,class_0,class_1
0,00eed32682bb,0.5,0.5
1,010ebe33f668,0.5,0.5
2,02fa521e1838,0.5,0.5
3,040e15f562a2,0.5,0.5
4,046e85c7cc7f,0.5,0.5


In [69]:
# y_pred_tr = np.array(submission.loc[:,['class_0','class_1']])
# print(ScoreMetric(y_true,y_pred_tr[:,1]))

# # 0.012554638990441243 vs 0.008646540967229121
# # 0.0020669455679357476

# #0.0045408272392828
# #0.0018594632297900106

# #0.018631567936169137
# #0.023465416212992158